# Create and Destroy Device

__Device creation, vector operation and torch interop, destroying device__

Vulky is a python facade to vulkan with reduced and simpler interface focused primarly to academic purposes. Objects are designed to represent graphics pipelines and techniques in a more compact way.

Let's start importing the module ```vulky```

In [1]:
import vulky as vk

Apart from rendering, vulky has objects to operate with vectors and matrices wrapping internally a torch tensor. This facilitates the interpretation of torch tensors as 3D objects and preserves differentiability when possible. This types also serves to declare equivalent vulkan shader types (```vec2```,...,```vec4```, ```mat2```, ..., ```mat4x3```, ```mat4```) 

In [2]:
a = vk.vec3(1., 0., .0)
b = vk.mat3.rotation(vk.vec3(0., 0.0, 1.0), 3.14159/2)
t = b@a
print(t)

vec3([1.2676e-06, 1.0000e+00, 0.0000e+00])


This vector and matrices types works also in batches. For instance, if you want to refer to a bidirectional array of (16,16) ```vec4``` you may use:

In [3]:
t = vk.vec4.zero(16, 16)
print(t)

vec4([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
   

Important observation is that indexing vectors and matrices refers to the component of the vector, not the instance of the batch, i.e.: 

In [4]:
t[0] = 1.0  # first component of all (16,16) vec4 where set to 1.0
t.y = 2.0  # equivalent to index, a named access to the field is also valid and refers to whole the batch
print(t)  

vec4([[[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.]],

        [[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.]],

        [[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.]],

        ...,

        [[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.]],

        [[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.]],

        [[1., 2., 0., 0.],
         [1., 2., 0., 0.],
         [1., 2., 0., 0.],
         ...,
   

For graphics, internally, vulky works with one single vulkan instance at the time but might work with several devices. There is a concept of active device and most of the methods of vulky library refers implicitly to that device. The active device can be selected with the ```device_manager``` method passing the device object. By default, the creation of a device, makes that device the active one. Notice, it is not a problem not to save the device if there is no intention to switch between devices in a future.

In [5]:
vk.create_device(debug=True)

[INFO] Vulkan Instance created...
[INFO] Debug instance created...
[INFO] Supported features {'coop_mat': True, 'rt': 1, 'rq': True, 'atomic': 1}
[INFO] Available devices: 1
[INFO] Selected device: NVIDIA GeForce RTX 2080 Ti

[INFO] Selected device index: 0

[INFO] 5 available queue family
[PRINT] Inserted device layer "VK_LAYER_KHRONOS_validation" (C:\VulkanSDK\1.3.275.0\Bin\.\VkLayer_khronos_validation.dll)
[PRINT] Inserted device layer "VK_LAYER_OBS_HOOK" (C:\ProgramData\obs-studio-hook\.\graphics-hook64.dll)
[PRINT] Inserted device layer "VK_LAYER_NV_optimus" (C:\WINDOWS\System32\DriverStore\FileRepository\nv_dispi.inf_amd64_cc569e59ca39c5fe\.\nvoglv64.dll)
[PRINT] vkCreateDevice layer callstack setup to:
[PRINT]    <Application>
[PRINT]      ||
[PRINT]    <Loader>
[PRINT]      ||
[PRINT]    VK_LAYER_NV_optimus
[PRINT]            Type: Implicit
[PRINT]                Disable Env Var:  DISABLE_LAYER_NV_OPTIMUS_1
[PRINT]            Manifest: C:\WINDOWS\System32\DriverStore\FileReposi

Vulky manages automatically two types of memory, the memory compatible with the host (CPU) and the memory purely in the graphic device (GPU). If cuda is present, the device memory is exported to cuda and Pytorch library, making the creation of tensors managed by vulkan simpler. 

In [6]:
t = vk.tensor(2,4)
print(t + 0.2)

tensor([[0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000]], device='cuda:0')


Although vulky tensors can be operated as regular ```torch``` tensors, the memory is owned by vulky and they must be deleted before vulkan device is destroyed.

In [7]:
del t
vk.quit()

[INFO] Destroyed vulkan instance
